In [1]:
import numpy as np
import cv2
import tensorflow as tf
import os

import matplotlib.pyplot as plt

from tensorflow import keras 
from tensorflow.keras import layers

In [2]:
my_model = tf.keras.models.load_model('model_face_emotions_detection.h5')

In [3]:
face_detect = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')

In [4]:
class_labels = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

In [5]:
def draw_border(img, pt1, pt2, color, thickness, r, d):
    x1,y1 = pt1
    x2,y2 = pt2
    # Top left
    cv2.line(img, (x1 + r, y1), (x1 + r + d, y1), color, thickness)
    cv2.line(img, (x1, y1 + r), (x1, y1 + r + d), color, thickness)
    cv2.ellipse(img, (x1 + r, y1 + r), (r, r), 180, 0, 90, color, thickness)
    # Top right
    cv2.line(img, (x2 - r, y1), (x2 - r - d, y1), color, thickness)
    cv2.line(img, (x2, y1 + r), (x2, y1 + r + d), color, thickness)
    cv2.ellipse(img, (x2 - r, y1 + r), (r, r), 270, 0, 90, color, thickness)
    # Bottom left
    cv2.line(img, (x1 + r, y2), (x1 + r + d, y2), color, thickness)
    cv2.line(img, (x1, y2 - r), (x1, y2 - r - d), color, thickness)
    cv2.ellipse(img, (x1 + r, y2 - r), (r, r), 90, 0, 90, color, thickness)
    # Bottom right
    cv2.line(img, (x2 - r, y2), (x2 - r - d, y2), color, thickness)
    cv2.line(img, (x2, y2 - r), (x2, y2 - r - d), color, thickness)
    cv2.ellipse(img, (x2 - r, y2 - r), (r, r), 0, 0, 90, color, thickness)


In [6]:
face_detect = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
#eye_detect = cv2.CascadeClassifier('haarcascade_eye.xml')

class_labels = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']

def face_detection(img,size=0.5):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face_roi = face_detect.detectMultiScale(img_gray, 1.3,1)
    if face_roi is ():
        return img

    for(x,y,w,h) in face_roi:
        x = x - 5
        w = w + 10
        y = y + 7
        h = h + 2
        draw_border(img, (x,y),(x+w,y+h),(0,0,204), 2,15, 10)
        #cv2.imshow('detected_face', img)
        #cv2.waitKey(0)
        img_gray_crop = img_gray[y:y+h,x:x+w]
        img_color_crop = img[y:y+h,x:x+w]
        
        
        final_image = cv2.resize(img_color_crop, (224,224))
        final_image = np.expand_dims(final_image, axis = 0)
        final_image = final_image/255.0
    
        prediction = my_model.predict(final_image)
        label=class_labels[prediction.argmax()]
        cv2.putText(frame,label, (x+20, y-40), cv2.FONT_HERSHEY_SCRIPT_COMPLEX,2, (92,79,19),3)
        #eye_roi = eye_detect.detectMultiScale(img_gray_crop)
        
        #for(ex,ey,ew,eh) in eye_roi:
            #cv2.rectangle(img_color_crop, (ex,ey),(ex+ew,ey+eh),(125,125,10), 2)
            #cv2.imshow('detected_face', img)

    img_color_crop = cv2.flip(img_color_crop, 1)
    return img

cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    cv2.imshow('LIVE', face_detection(frame))
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()